In [1]:
# Set up CUDA
#First Change runtime to GPU and run this cell
!pip install git+https://github.com/afnan47/cuda.git
%load_ext nvcc_plugin


  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-zrwcpj00
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-zrwcpj00
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4290 sha256=40d78be52d979d31b28f349bd42ba4a4267d7c4e60cab447851b767a7c617954
  Stored in directory: /tmp/pip-ephem-wheel-cache-r3iyed5q/wheels/bc/4e/e0/2d86bd15f671dbeb32144013f1159dba09757fde36dc51a963
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [9]:
pip install pycuda


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 9.1 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp311-cp311-linux_x86_64.whl size=660425 sha256=23249956253b77b3813008da093ca06dd43da9934d86272e7f1763f71e0274ec
  Stored in directory: /root/.cache/pip/wheels/77/7e/6c/d2d1451ea6424cdc3d67b36c16fa7111eafdf2034bc3405666
Successfully built pycuda


In [10]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
from pycuda.compiler import SourceModule

# Define CUDA kernel for addition
kernel_code = """
__global__ void add(int *A, int *B, int *C, int size)
{
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid < size)
    {
        C[tid] = A[tid] + B[tid];
    }
}
"""

# Initialize vectors with random values
def initialize(size):
    return np.random.randint(0, 10, size).astype(np.int32)

# Print vector
def print_vector(vec):
    print(" ".join(map(str, vec)))

# Vector size
N = 4

# Initialize vectors A and B
A = initialize(N)
B = initialize(N)

# Print vectors A and B
print("Vector A:", A)
print("Vector B:", B)

# Allocate memory on the device
A_gpu = cuda.mem_alloc(A.nbytes)
B_gpu = cuda.mem_alloc(B.nbytes)
C_gpu = cuda.mem_alloc(A.nbytes)

# Copy input vectors to the device
cuda.memcpy_htod(A_gpu, A)
cuda.memcpy_htod(B_gpu, B)

# Compile the CUDA code
mod = SourceModule(kernel_code)

# Get the kernel function
add_kernel = mod.get_function("add")

# Define number of threads and blocks
threads_per_block = 256
blocks_per_grid = (N + threads_per_block - 1) // threads_per_block

# Launch the CUDA kernel
add_kernel(A_gpu, B_gpu, C_gpu, np.int32(N), block=(threads_per_block, 1, 1), grid=(blocks_per_grid, 1))

# Copy the result from device to host
C = np.empty_like(A)
cuda.memcpy_dtoh(C, C_gpu)

# Print the result
print("Addition result:", C)



Vector A: [4 5 7 0]
Vector B: [2 9 2 5]
Addition result: [ 6 14  9  5]
